In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import random
import struct
import matplotlib.pyplot as plt
import parameters as var #Configuration and coarsening parameters
var.init() #initializes parameters
import utils as ut #Some utility functions 
import loss_function as lf #Custom loss function

import operators_torch as op #Interpolator and prolongator given a set of test vectors
from opendataset import ConfsDataset #class for opening gauge confs
import model as mod #import machine learning model

from torch.profiler import profile, ProfilerActivity, record_function


#--------Most likely I will need some of these in the future-------#
#import torch.optim as optim
#import torchvision.transforms as transforms
#import torchvision.utils as vutils
var.print_parameters()
device = (
    "cuda"
    if var.NGPU>0
    else "cpu"
)
print(f"Using {device} device")

*********** Configuration parameters ***********
* β=2, Nx=32, Nt=32
* Lattice sites=2048
* m0=-0.1884
* blocks_x=2, blocks_t=2 (for the aggregation)
* Nv=20
* Number of confs=1000
* Confs used for training=900
************************************************
Using cuda device


In [2]:
"""
Loading the configurations and the near-kernel test vectors
We split train and test set
"""
dataset = ConfsDataset()                     
total_len = len(dataset)                    
train_len = int(var.TRAIN_PROP * total_len) 
test_len  = total_len - train_len   
torch.manual_seed(42)                       # <-- any integer you like

train_set, test_set = torch.utils.data.random_split(
    dataset,
    [train_len,  test_len]          # lengths in the same order
)

workers    = 2
# Batch size
batch_size = 100

#train dataloader
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,          # we usually want shuffling *only* for training
    num_workers=workers,
    pin_memory=True
)

#test dataloader
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=workers,
    pin_memory=True
)

device = var.DEVICE

"""
Custom weights initialization
"""  
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02) #nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0) #nn.init.constant_(m.bias.data, 0.0)

In [3]:
"""
Declare the model
"""
model = mod.TvGenerator(var.NGPU,batch_size).to(device)
if (device.type == 'cuda') and (var.NGPU > 1):
    model = nn.DataParallel(model, list(range(var.NGPU)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
model.apply(weights_init)

TvGenerator(
  (main): Sequential(
    (0): Conv2d(4, 128, kernel_size=(2, 2), stride=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=128)
    (3): Conv2d(128, 64, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): PReLU(num_parameters=64)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=4096, out_features=256, bias=True)
    (8): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): PReLU(num_parameters=256)
    (10): Linear(in_features=256, out_features=64, bias=True)
    (11): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): PReLU(num_parameters=64)
    (13): Linear(in_features=64, out_features=81920, bias=True)
  )
)

In [6]:
activities = [ProfilerActivity.CPU,ProfilerActivity.CUDA]
device = "cuda" if torch.cuda.is_available() else ""
sort_by_keyword = device + "_time_total"
criterion = lf.CustomLossTorch().to(device)  
with profile(activities=activities, record_shapes=True) as prof:
    with record_function("model_inference"):
        for gconfs, near_kernel, ID in train_loader:
            gconfs, near_kernel = gconfs.to(device), near_kernel.to(device)
            
            pred = model(gconfs).view(batch_size, var.NV, 4, var.NT, var.NX) 
            real = torch.stack([pred[:,:, 0], pred[:,:, 1]], dim=2)   # (B,NV,2,NT,NX)
            imag = torch.stack([pred[:,:, 2], pred[:,:, 3]], dim=2)   # (B,NV,2,NT,NX)
            pred_complex = torch.complex(real, imag)                  # (B,NV,2,NT,NX)

            #Normalizing the fake test vectors
            norms = torch.linalg.vector_norm(pred_complex[:,:],dim=(-3,-2, -1)).view(batch_size, var.NV, 1, 1, 1)
            norms_broadcastable = norms.view(batch_size, var.NV, 1, 1, 1)
            pred_complex_normalized = pred_complex / norms_broadcastable
       
            #We assemble P, P^+ with the SAP test vectors and find other vectors that are similar to them.
            loss = criterion(near_kernel, pred_complex_normalized)   # loss is a scalar Tensor
            break
print(prof.key_averages().table(sort_by=sort_by_keyword, row_limit=500))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us        1.568s       178.52%        1.568s        1.568s             1  
                                        model_inference        20.70%     406.441ms       100.00%        1.964s        1.964s       0.000us         0.00%     878.536ms     878.536ms             1  
         

In [ ]:
# convert.py
#from glob import glob
#from opendataset import read_binary_conf

PATH = '/wsgjsc/home/nietocastellanos1/Documents/SchwingerModel/fermions/SchwingerModel' + \
        '/confs/b{0}_{1}x{2}/{3}'.format(int(var.BETA),var.NX,var.NT,var.M0_FOLDER)

#PATH='sap/near_kernel/b{0}_{1}x{2}/{3}'.format(int(var.BETA),var.NX,var.NT,var.M0_FOLDER)

conf_files = []
for path in sorted(glob(PATH+"/*.ctxt")):
    conf_files.append(path)
    #arr = read_binary_conf("",path)
    #torch.save(torch.from_numpy(arr), path.replace(".tv", ".pt"))

In [ ]:
idx = 0
conf_file = conf_files[idx]
gauge_conf = read_binary_conf(None,conf_file)
#tvectors = torch.zeros(var.NV,2,var.NT,var.NX,dtype=torch.complex128)

#gauge_conf = torch.load(self.conf_files[idx].replace('.ctxt','.pt'), mmap=True)
#tvectors = torch.load(self.tv_files[idx].replace('.tv','.pt'), mmap=True)
        
real = torch.tensor(np.real(gauge_conf), dtype=torch.float32)
imag = torch.tensor(np.imag(gauge_conf), dtype=torch.float32)
gauge_conf = torch.tensor(np.array([real,imag])).reshape(4, var.NT, var.NX)  
        
#for tvID in range(var.NV):
#    vector = read_binary_conf(None,tv_files[idx][tvID])
#    real = torch.tensor(np.real(vector), dtype=torch.float64)
#    imag = torch.tensor(np.imag(vector), dtype=torch.float64)
#    tvectors[tvID] = torch.complex(real,imag).reshape(2, var.NT, var.NX) 

In [ ]:
gauge_conf_2 = torch.load(conf_file.replace('.ctxt','.pt'), mmap=True,weights_only=True)
gauge_conf_2 = torch.cat([torch.real(gauge_conf_2), torch.imag(gauge_conf_2)], dim=0).float()   # [4, NT, NX]

In [ ]:
gauge_conf_2

In [ ]:
print(gauge_conf[0,0,0],gauge_conf[2,0,0])
print(gauge_conf[1,0,0],gauge_conf[3,0,0])

print(gauge_conf_2[0,0,0],gauge_conf_2[2,0,0])
print(gauge_conf_2[1,0,0],gauge_conf_2[3,0,0])

In [ ]:
tvectors = torch.load("sap/near_kernel/b2_32x32/m-018/tvector_32x32_b20000_m-01884_nconf264_tv12.tv".replace('.tv','.pt'), mmap=True,weights_only=True)